# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [64]:
# Dependencies and Setup
import pandas as pd
import requests
import numpy as np
from pprint import pprint
import json
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import requests
# Data Viz
import cartopy
import geoviews
import pyproj



# Import API key
from api_keys import geoapify_key

In [65]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [66]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [67]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 10
max_temp = 80
max_wind = 10

# Drop any rows with null values
df2 = city_data_df.dropna()

mask = (df2["Max Temp"] >= min_temp) & (df2["Max Temp"] < max_temp) & (df2["Wind Speed"] < max_wind)
df2 = df2.loc[mask].reset_index(drop=True)

# Display sample data
df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
4,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
...,...,...,...,...,...,...,...,...,...,...
395,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330
396,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563
397,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565
398,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566


### Step 3: Create a new DataFrame called `hotel_df`.

In [68]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
5,ushuaia,AR,-54.8000,-68.3000,81,
6,albany,US,42.6001,-73.9662,74,
7,carnarvon,AU,-24.8667,113.6333,76,
8,sisimiut,GL,66.9395,-53.6735,98,
9,saint-pierre,RE,-21.3393,55.4781,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [71]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Hotel SØMA
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hotel f

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [72]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)